In [3]:
import ollama
import requests
import json

In [4]:
model = "llama3.1"

#### Task 1: Interact with deployed LLM via python 


**Objective:**

Explore different techniques to interact with the deployed LLM.

**Task Description:**

1. Use Request libaray (HTTP Client) -> [How To](https://requests.readthedocs.io/en/latest/user/quickstart/#make-a-request)
2. User Ollama python library -> [How To](https://pypi.org/project/ollama/)

In [5]:
# Simple HTTP Request via requests

# Define the URL of the deployed LLM
url = "http://localhost:11434/api/generate"

# Define the prompt
body = {
    "model": model,
    "prompt": "Describe Generative AI in two sentences."
}

# Send the POST request
response = requests.post(url, json=body)

# Check if the request was successful
if response.status_code == 200:
    # Process the response
    response_text = response.text

    # Convert each line to json
    response_lines = response_text.splitlines()
    response_json = [json.loads(line) for line in response_lines]
    for line in response_json:
        # Print the response. No line break
        print(line["response"], end="")
else:
    print("Error:", response.status_code)


Here is a two-sentence description of Generative AI:

Generative AI refers to a subset of artificial intelligence that uses algorithms and statistical models to generate new, original content such as images, videos, music, text, or even 3D models that mimic the style of existing works or simulate real-world phenomena. This type of AI can create unique outputs without being explicitly programmed to do so, often by learning from large datasets and using complex mathematical techniques like deep learning and neural networks.

In [6]:
# Simple API Call via ollama

response = ollama.chat(model=model, messages=[
  {
    "role": "user",
    "content": "Describe Generative AI in two sentences.",
  },
])
print(response["message"]["content"])

Generative AI refers to a type of artificial intelligence that can generate new, original content such as text, images, audio, or videos based on patterns and structures learned from existing data. This technology uses machine learning algorithms to create novel outputs that are often indistinguishable from those created by humans, with applications in fields like art, music, writing, and even product design.


In [7]:
# Streaming API Call via ollama

# Response streaming can be enabled by setting stream=True, 
# modifying function calls to return a Python generator where each part is an object in the stream.

stream = ollama.chat(
    model=model,
    messages=[{"role": "user", "content": "Describe Generative AI in 5 sentences."}],
    stream=True,
)

for chunk in stream:
  print(chunk["message"]["content"], end="", flush=True)

Here is a description of Generative AI:

Generative AI refers to a type of artificial intelligence that can create new, original content such as images, videos, music, or text based on patterns and structures it has learned from existing data. These models use complex algorithms and machine learning techniques to generate novel outputs that are often indistinguishable from those created by humans. Generative AI has various applications, including art, design, entertainment, and even tasks like language translation and text summarization. Some notable examples of generative AI include the development of AI-generated music and art, such as portraits and landscapes, which have sparked both wonder and controversy in recent years. Overall, Generative AI represents a significant advancement in the field of artificial intelligence and has the potential to transform various industries and aspects of society.

#### Task 2: Experimenting with Prompt Techniques

**Objective:**

Objective: Explore different prompt techniques (Zero Shot, One Shot, and Few Shot) by sending different types of prompts to the LLM.

![image](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*QSpK--jqPiUU_OHuZvtUWA.png)

**Task Description:**

1. Create three prompts: a Zero Shot prompt, a One Shot prompt, and a Few Shot prompt.
2. Send these prompts to the LLM and observe the differences in the responses.
3. Compare and discuss the responses.

In [8]:
# Define the prompts
zero_shot_prompt = "Classify the sentiment of the following review as Positive, Neutral, or Negative: 'I loved this movie!' Sentiment:"

one_shot_prompt = """Classify the sentiment of the following review as Positive, Neutral, or Negative:
Review: 'I loved this movie!'
Sentiment: Positive

Classify the sentiment of the following review:
Review: 'I hate the chair.'
Sentiment:"""

few_shot_prompt = """Classify the sentiment of the following reviews as Positive, Neutral, or Negative:
Review: 'I loved this movie!'
Sentiment: Positive

Review: 'I hate the chair!'
Sentiment: Negative

Classify the sentiment of the following review:
Review: 'Who would use this product?'
Sentiment:
"""

# Stream the responses and print them
for idx, prompt in enumerate([zero_shot_prompt, one_shot_prompt, few_shot_prompt]):
    prompt_type = ["Zero-Shot", "One-Shot", "Few-Shot"][idx]
    print(f"\n--- {prompt_type} Prompt ---\n")
    print(f"User Prompt:\n{prompt}\n")
    
    stream = ollama.chat(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        stream=True,
    )
    
    print("Model Output:")
    for chunk in stream:
        print(chunk["message"]["content"], end="", flush=True)
    print("\n-----------------------------\n")



--- Zero-Shot Prompt ---

User Prompt:
Classify the sentiment of the following review as Positive, Neutral, or Negative: 'I loved this movie!' Sentiment:

Model Output:
Sentiment: **Positive**
-----------------------------


--- One-Shot Prompt ---

User Prompt:
Classify the sentiment of the following review as Positive, Neutral, or Negative:
Review: 'I loved this movie!'
Sentiment: Positive

Classify the sentiment of the following review:
Review: 'I hate the chair.'
Sentiment:

Model Output:
Based on the review "I hate the chair.", I would classify the sentiment as:

Sentiment: Negative
-----------------------------


--- Few-Shot Prompt ---

User Prompt:
Classify the sentiment of the following reviews as Positive, Neutral, or Negative:
Review: 'I loved this movie!'
Sentiment: Positive

Review: 'I hate the chair!'
Sentiment: Negative

Classify the sentiment of the following review:
Review: 'Who would use this product?'
Sentiment:


Model Output:
The sentiment of the given review is: 

#### Task 3: Prompt Refinement and Optimization

**Objective:** 

Refine a prompt to improve the clarity and quality of the LLM's response.

**Task Description:**

- Start with a basic prompt asking the LLM to summarize a paragraph.
- Refine the prompt by adding specific instructions to improve the summary's quality.

In [9]:
# Original prompt
original_prompt = "Summarize the following paragraph: Generative AI is a field of artificial intelligence focused on creating new content based on patterns learned from existing data. It has applications in text, image, and music generation, and is increasingly being used in creative industries."

# Refined prompt
refined_prompt = "Provide a concise and informative summary of the following paragraph, highlighting its key points: Generative AI is a field of artificial intelligence focused on creating new content based on patterns learned from existing data. It has applications in text, image, and music generation, and is increasingly being used in creative industries."

# Stream the responses and print them
for idx, prompt in enumerate([original_prompt, refined_prompt]):
    prompt_type = ["Original Prompt", "Refined Prompt"][idx]
    print(f"\n--- {prompt_type} ---\n")
    print(f"User Prompt:\n{prompt}\n")
    
    stream = ollama.chat(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        stream=True,
    )
    
    print("Model Output:")
    for chunk in stream:
        print(chunk["message"]["content"], end="", flush=True)
    print("\n-----------------------------\n")



--- Original Prompt ---

User Prompt:
Summarize the following paragraph: Generative AI is a field of artificial intelligence focused on creating new content based on patterns learned from existing data. It has applications in text, image, and music generation, and is increasingly being used in creative industries.

Model Output:
Here is a summary of the paragraph:

Generative AI creates new content (such as text, images, or music) by learning patterns from existing data.
-----------------------------


--- Refined Prompt ---

User Prompt:
Provide a concise and informative summary of the following paragraph, highlighting its key points: Generative AI is a field of artificial intelligence focused on creating new content based on patterns learned from existing data. It has applications in text, image, and music generation, and is increasingly being used in creative industries.

Model Output:
Here is a concise and informative summary of the paragraph:

**Key Points:** 

* Generative AI cr

#### Task 4: Structured Prompting with Roles (Pirate Theme)

**Objective:**

Learn how to use structured prompts that combine role assignment, clear instructions, and examples to improve the output of language models. In this task, you will guide the AI to respond as a pirate who is also an expert in machine learning.

**Instructions:**

Role Assignment: In your prompt, specify the role of the AI as a Machine Learning Expert who speaks like a pirate.
Instruction: Clearly state what you want the AI to explain or discuss in pirate language.
Examples: Provide examples to guide the AI in using pirate lingo while explaining technical concepts.

In [10]:
# Combined Techniques Prompt with Pirate Theme
structured_prompt = """
Role: Ye be a Machine Learning Expert who talks like a pirate.

Instruction: Explain the followin' machine learnin' concepts in a way that a pirate would.

Example 1:
Question: What be supervised learnin'?
Answer: Arrr, supervised learnin' be when ye have a chest o' data, and fer each piece o' data, ye know the treasure map that shows ye the right path. Ye train the model until it knows how to follow the map on its own, savvy?

Example 2:
Question: What be overfittin'?
Answer: Ahoy, overfittin' be when yer model gets too cozy with yer trainin' data, like a pirate who knows only one sea. It performs well on the known waters but fails when ye set sail on new seas, matey!

Example 3:
Question: What be a neural network?
Answer: Avast! A neural network be like a crew o' shipmates, each one passin' information along, makin' decisions, and workin' together to find the best course to the treasure, arr!

Explain this:
Question: What be unsupervised learnin'?
Answer:
"""

# Stream the response and print it
print("=== User Prompt ===")
print(structured_prompt)

stream = ollama.chat(
    model=model,
    messages=[{"role": "user", "content": structured_prompt}],
    stream=True,
)

print("\n=== Model Output ===")
for chunk in stream:
    print(chunk["message"]["content"], end="", flush=True)
print("\n")


=== User Prompt ===

Role: Ye be a Machine Learning Expert who talks like a pirate.

Instruction: Explain the followin' machine learnin' concepts in a way that a pirate would.

Example 1:
Question: What be supervised learnin'?
Answer: Arrr, supervised learnin' be when ye have a chest o' data, and fer each piece o' data, ye know the treasure map that shows ye the right path. Ye train the model until it knows how to follow the map on its own, savvy?

Example 2:
Question: What be overfittin'?
Answer: Ahoy, overfittin' be when yer model gets too cozy with yer trainin' data, like a pirate who knows only one sea. It performs well on the known waters but fails when ye set sail on new seas, matey!

Example 3:
Question: What be a neural network?
Answer: Avast! A neural network be like a crew o' shipmates, each one passin' information along, makin' decisions, and workin' together to find the best course to the treasure, arr!

Explain this:
Question: What be unsupervised learnin'?
Answer:


=== M